In [ ]:
import time
import requests
from bs4 import BeautifulSoup
import re
import json
import sys
from datetime import datetime
import numpy as np
stockid_h=np.load('stockid_h.npy').tolist()
stockid_s=np.load('stockid_s.npy').tolist()
stockid=stockid_s+stockid_h
import pandas as pd
stocks_info=pd.DataFrame(columns=['time','total_reads'],index=stockid)
#print(stocks_info)

In [ ]:


start_page=1
end_page=6

for this_id in stockid:
    base_url='http://guba.eastmoney.com/list,'+this_id+'_{}.html'
    total_reads=0
    total_time_stamp=[]
    #print('curr stock_id')
    #print(this_id)
    #print ("\r ".format(this_id),end="")

    for page in range(start_page,end_page):
        try:
    
            #time.sleep(1)
            #print('cur page: ',page) 
            r=requests.get(base_url.format(page))
            #print(r.status_code)
            #print(r.content)
            article_list = BeautifulSoup(r.content, 'lxml').find_all(class_='articleh normal_post')
            
            #print('item_count: ')
            for item in article_list:
                reads=item.find_all('span',class_='l1 a1')[0].get_text()
                if '万' in reads:
                    reads=float(reads[:-1])*10000
                else:
                    reads=float(reads)
                total_reads+=reads
                time_stamp=item.find_all('span',class_='l5 a5')[0].get_text()
                total_time_stamp.append(time_stamp)
                #print(time_stamp)
        except:
            print('error')
 
    stocks_info.loc[this_id]=[total_time_stamp,total_reads]

#stocks_info.to_csv('./stocks_frequency_5.csv')

In [ ]:

#stocks_info=pd.read_csv('./stocks_frequency_5.csv',index_col=0)
#print(stocks_info.describe())
stocks_info=stocks_info.drop_duplicates()
#print(stocks_info.describe())
idx=stocks_info.index.tolist()
#print(len(idx))
#print(len(list(set(idx))))

In [ ]:
import ast
from datetime import datetime

new_info=pd.DataFrame(columns=['total_reads','counts','deltatime','density','RxD'],index=stocks_info.index)
for index,row in stocks_info.iterrows():
    x=ast.literal_eval(row['time'])#字符串类型list转list
    counts=len(x)
    #print(x[0])
    #注意时间反了
    start_time=datetime.strptime( '2019-'+x[0],'%Y-%m-%d %H:%M')
    end_time=datetime.strptime( '2019-'+x[-1],'%Y-%m-%d %H:%M')
    deltatime=(start_time-end_time).total_seconds()
    density=counts*1000/deltatime
    print(index,row['total_reads'],counts,deltatime,density)
    new_info.loc[index]=[row['total_reads'],counts,deltatime,density,row['total_reads']*density]

new_info=new_info.sort_values(by="RxD" , ascending=False)
new_info.to_csv('./stocks_info_final_2.csv') 